In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [5]:
# Skip date
df1 = pd.read_csv('combined_data_1.txt', header = None, names = ['User_Id', 'Rating','Date'])

df1['Rating'] = df1['Rating'].astype(float)

print('Dataset 1 shape: {}'.format(df1.shape))
print('-Dataset examples-')
print(df1.iloc[::5000000, :])#START:STOP:STEP (5000000)

Dataset 1 shape: (24058263, 3)
-Dataset examples-
          User_Id  Rating        Date
0              1:     NaN         NaN
5000000   2560324     4.0  2005-12-06
10000000  2271935     2.0  2005-04-11
15000000  1921803     2.0  2005-01-31
20000000  1933327     3.0  2004-11-10


In [6]:
df1.index = np.arange(0,len(df1))
# get movie count (where rating ==NaN)
movie_count = df1.isnull().sum()[1] #df.isnull() searches for NaN values across all columns, we get in [1] column-rating

# get customer count (amount of unique user_id-movie_id)
user_count = df1['User_Id'].nunique() - movie_count

# get rating count (for ratings not only unique, all values except movies ids)
rating_count = df1['User_Id'].count() - movie_count

movie_count, user_count , rating_count

(4499, 470758, 24053764)

In [7]:
#Movie ID is really a mess import! 
#Looping through dataframe to add Movie ID column WILL make the Kernel run out of memory as it is too inefficient. 
#I achieve my task by first creating a numpy array with correct length 
#then add the whole array as column into the main dataframe!

#WORKING APPROXIMATELY 5 MIN

df_nan = pd.DataFrame(pd.isnull(df1.Rating)) #new df with indexes standard and one column "Rating" with True/False values
#df1.Rating gets indexes+"Rating" column values
#pd.isnull(df1.Rating) gets indexes+"Rating" mask column values (True=NaN, False-otherwise)

df_nan = df_nan[df_nan['Rating'] == True]#new df with old indexes and one column "Rating" with only True values
df_nan = df_nan.reset_index() # adds new indexes, old-moves to new column "index"


 # zip(df_nan['index'][1:],df_nan['index'][:-1]) returns iterator of tuples

movie_np = []
movie_id = 1 #we can start with 1 and sum it in loop,because all movie_id sorted in dataset from 1 to last (increase order)

# i-current movie_id j-previous movie_id , its needed for counting the difference because
#we need to know amount of rows=amount of ratings 
# so (i-j-1)=amount of not null ratings of whole users dataset for one movie (which has movie_id)
for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id) # 1st arg-shape, second-value add row , because movie_np is array 
    temp
    movie_np = np.append(movie_np, temp) # adds element to the end of array
    movie_id += 1 #we can sum it in loop дшлу +1 ,because all movie_id sorted in dataset from 1 to last (increase order)
    
# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df1) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)
#finally after loop we get array movie_np size=amount of non null ratings, at each index it has corresponding 
#movie_id value for tis rating +can be concatenated with cleaned df
print('Length: {}'.format(len(movie_np)))

Length: 24053764


In [8]:
# remove those Movie ID rows from initial df (rows where rating=NaN)
df1 = df1[pd.notnull(df1['Rating'])]

df1['Movie_Id'] = movie_np.astype(int) #Add movie_id column in df as movie_np received on previous step
df1['User_Id'] = df1['User_Id'].astype(int)
print('-Dataset examples-')
print(df1.iloc[::5000000, :])

-Dataset examples-
          User_Id  Rating        Date  Movie_Id
1         1488844     3.0  2005-09-06         1
5000996    501954     2.0  2004-08-26       996
10001962   404654     5.0  2005-08-29      1962
15002876   886608     2.0  2005-09-19      2876
20003825  1193835     2.0  2003-08-13      3825


In [11]:
# Get one hot encoding of column User_id
#one_hot_users = pd.get_dummies(df1['User_Id'])
#one_hot_users
# Drop column User_Id as it is now encoded
#df1 = df1.drop('User_Id',axis = 1)
# Join the encoded df
#df1 = df1.join(one_hot_users)
#df1  

#This approach with dummies wouldnt work, because dataset is too large

In [31]:
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 
encoder = OneHotEncoder()
#X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']] 
#X-train matrix for encoder to learn categories
#enc.fit(X)  #process of encoder learning categories
one_hot_user_matrix = encoder.fit_transform(np.asarray(df1['User_Id']).reshape(-1,1)) 
print(one_hot_user_matrix[:, :]) #amount of rows=amount of ratings , amount of columns=amount of unique user_ids
#enc.transform([['female', 'from US', 'uses Safari'],['male', 'from Europe', 'uses Safari']]).toarray()
#finally transforming test matrix into one-hot-SLIGHTLY dont understand the output
#array([[1., 0., 0., 1., 0., 1.],[0., 1., 1., 0., 0., 1.]])

#df1 = df1.join(one_hot_user_matrix)
#print(df1.iloc[::5000000, :])

C:\Users\Veronika_Revyakina\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


  (0, 264791)	1.0
  (1, 146632)	1.0
  (2, 157756)	1.0
  (3, 5365)	1.0
  (4, 146873)	1.0
  (5, 159340)	1.0
  (6, 22077)	1.0
  (7, 222084)	1.0
  (8, 327686)	1.0
  (9, 397671)	1.0
  (10, 267593)	1.0
  (11, 392334)	1.0
  (12, 460236)	1.0
  (13, 425)	1.0
  (14, 96982)	1.0
  (15, 215175)	1.0
  (16, 143503)	1.0
  (17, 193552)	1.0
  (18, 304537)	1.0
  (19, 66354)	1.0
  (20, 192402)	1.0
  (21, 221658)	1.0
  (22, 99569)	1.0
  (23, 384788)	1.0
  (24, 210278)	1.0
  :	:
  (24053739, 192931)	1.0
  (24053740, 233027)	1.0
  (24053741, 107764)	1.0
  (24053742, 54650)	1.0
  (24053743, 237582)	1.0
  (24053744, 188967)	1.0
  (24053745, 329447)	1.0
  (24053746, 47830)	1.0
  (24053747, 420612)	1.0
  (24053748, 94419)	1.0
  (24053749, 78800)	1.0
  (24053750, 371923)	1.0
  (24053751, 99100)	1.0
  (24053752, 54078)	1.0
  (24053753, 116865)	1.0
  (24053754, 48566)	1.0
  (24053755, 32896)	1.0
  (24053756, 394458)	1.0
  (24053757, 319568)	1.0
  (24053758, 455348)	1.0
  (24053759, 460485)	1.0
  (24053760, 318604)	